In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns',None)
pd.options.mode.chained_assignment = None

In [2]:
rollingAvg = 20
stoplossPerc = 0.5
stdMultiplier = 1

In [3]:
niftySingle = pd.read_csv('niftySingle.csv')
niftySingle = niftySingle.iloc[:,:5]
suffix = 'nifty'
niftySingle = niftySingle.rename(columns={'Date':'date','Open': f'{suffix}Open', 'High':f"{suffix}High", 'Low': f"{suffix}Low", 'Close':f'{suffix}Close'
})

niftySingle['date'] = pd.to_datetime(niftySingle['date'])
niftySingle.set_index('date',inplace=True)
niftySingle

,niftyOpen,niftyHigh,niftyLow,niftyClose
date,,,,
2012-01-03,4675.799805,4773.100098,4675.799805,4765.299805
2012-01-04,4774.950195,4782.850098,4728.850098,4749.649902
2012-01-05,4749.000000,4779.799805,4730.149902,4749.950195
2012-01-06,4724.149902,4794.899902,4686.850098,4754.100098
2012-01-09,4747.549805,4758.700195,4695.450195,4742.799805
...,...,...,...,...
2023-04-26,17767.300781,17827.750000,17711.199219,17813.599609
2023-04-27,17813.099609,17931.599609,17797.900391,17915.050781
2023-04-28,17950.400391,18089.150391,17885.300781,18065.000000


In [5]:
# list_of_companies = os.listdir('C:\\keshav\\bollingerBand\\Nifty50_Data')
# list_of_companies

# list_of_companies[19]

In [4]:
list_of_companies= ['RELIANCE.NS.csv', 'HDFCBANK.NS.csv', 'ICICIBANK.NS.csv', 'INFY.NS.csv', 'HDFC.NS.csv', 'TCS.NS.csv', 'ITC.NS.csv', 'KOTAKBANK.NS.csv', 'LT.NS.csv', 'AXISBANK.NS.csv']
list_of_companies

['RELIANCE.NS.csv',
 'HDFCBANK.NS.csv',
 'ICICIBANK.NS.csv',
 'INFY.NS.csv',
 'HDFC.NS.csv',
 'TCS.NS.csv',
 'ITC.NS.csv',
 'KOTAKBANK.NS.csv',
 'LT.NS.csv',
 'AXISBANK.NS.csv']

In [5]:
allStocks = []

for i in range(len(list_of_companies)):
        stockSingle = pd.read_csv(f'C:\\keshav\\bollingerBand\\Nifty50_Data\\{list_of_companies[i]}', usecols=['Date','Open',"High", "Low",'Close'])

        company_name = f"{list_of_companies[i].split('.')[0]}"

        stockSingle = stockSingle.rename(columns={'Date':'date','Open': f'{company_name}Open', 'High':f"{company_name}High", 'Low': f"{company_name}Low", 'Close':f'{company_name}Close'
        })

        stockSingle['date'] = pd.to_datetime(stockSingle['date'])
        stockSingle.set_index('date',inplace=True)

        mergedDf = niftySingle.merge(stockSingle,left_index=True, right_index=True)

        mergedDf['ratioOpen'] = mergedDf[f'{company_name}Open'] / mergedDf['niftyOpen']
        mergedDf['ratioHigh'] = mergedDf[f'{company_name}High'] / mergedDf['niftyHigh']
        mergedDf['ratioLow'] =  mergedDf[f'{company_name}Low'] / mergedDf['niftyLow']
        mergedDf['ratioClose'] = mergedDf[f'{company_name}Close'] / mergedDf['niftyClose']

        mergedDf['mean'] = mergedDf['ratioClose'].rolling(rollingAvg).mean()
        mergedDf['std'] = mergedDf['ratioClose'].rolling(rollingAvg).std()

        mergedDf[f"+{stdMultiplier}std"] = mergedDf['mean'] + stdMultiplier*mergedDf['std']
        mergedDf[f"-{stdMultiplier}std"] = mergedDf['mean'] - stdMultiplier*mergedDf['std']
        mergedDf.dropna(subset=['mean'], inplace=True)


        # calculating  bandTouch
        mask1 = mergedDf['ratioClose'] <= mergedDf['-1std']
        mask2 = mergedDf['ratioLow'] <= mergedDf['-1std']
        mask3 = mergedDf['ratioClose'] >= mergedDf['+1std']
        mask4 = mergedDf['ratioHigh'] >= mergedDf['+1std']

        mergedDf['bandTouch'] = np.where(mask1 | mask2,'Lower',
                                                        np.where(mask3 |  mask4, "Upper",None)
                                                )   


        #bandTouch1
        mergedDf['bandTouch1'] = mergedDf['bandTouch'].fillna(method='ffill')


        # closeLowerBand
        mergedDf['closeLowerBand'] = np.where(
                                        mergedDf['ratioClose'] >= mergedDf['-1std'],"Above","Below" 
                                                )
                


        #closeMidBand
        mergedDf['closeMidBand'] = np.where(
        mergedDf['ratioClose'] >= mergedDf['std'],
        "Above","Below" 
                )


        # closeUpperBand
        mergedDf['closeUpperBand'] = np.where(
        mergedDf['ratioClose'] >= mergedDf['+1std'],
        "Above","Below" 
                )


        #closeAboveBelow
        mask1 = pd.isna(mergedDf['bandTouch1'])
        mask2 = mergedDf['bandTouch1'] == 'Upper'
        mask3 = mergedDf['ratioClose'] <= mergedDf['+1std']
        mask4 = mergedDf['ratioClose'] >= mergedDf['-1std']

        mergedDf['closeAboveBelow'] = np.where(
                                                mask1,None,np.where(
                                                        mask2,
                                                                np.where(mask3,"Below", "Above" ),
                                                                np.where(mask4,"Above", "Below" ) )
                                                )


        #signal
        mergedDf['signal'] = None

        for i in range(mergedDf.shape[0]):
                if (mergedDf['bandTouch1'].iloc[i] == 'Upper') and (mergedDf['closeUpperBand'].iloc[i] == 'Below'):
                        mergedDf['signal'].iloc[i] = 'Sell'
                elif (mergedDf['bandTouch1'].iloc[i] == 'Lower') and (mergedDf['closeLowerBand'].iloc[i] == 'Above'):
                        mergedDf['signal'].iloc[i] = 'Buy'
                else:
                        mergedDf['signal'].iloc[i] = mergedDf['signal'].iloc[i-1]


        
        #entrySignal
        mergedDf['entrySignal'] = np.where(mergedDf['signal'] != mergedDf['signal'].shift(),mergedDf['signal'],None )


        #entryRatio
        mergedDf['entryRatio'] = np.nan
        for i in range(mergedDf.shape[0]):
                mergedDf['entryRatio'] = np.where(
                                                pd.notna(mergedDf['entrySignal']),
                                                mergedDf['ratioClose'],
                                                mergedDf['entryRatio'].shift()
                                                )

                mergedDf['entryRatio'] = mergedDf['entryRatio']


        #entryPrice
        mergedDf['entryPrice'] = np.nan
        for i in range(mergedDf.shape[0]):
                mergedDf['entryPrice'] = np.where(
                                                pd.notna(mergedDf['entrySignal']),
                                                mergedDf[f'{company_name}Close'],
                                                mergedDf['entryPrice'].shift()
                                                        )


        #stoploss
        mergedDf['stopLoss'] = np.nan
        for i in range(mergedDf.shape[0]):
                mergedDf['stopLoss'] = np.where(
                        pd.notna(mergedDf['entrySignal']),
                        stoplossPerc*mergedDf['std'],
                        mergedDf['stopLoss'].shift()

                )
      

        #stoplossRatio
        mergedDf['stopLossRatio'] = np.nan

        for i in range(mergedDf.shape[0]):
                mergedDf['stopLossRatio'] = np.where(
                                                        mergedDf['entrySignal'] == 'Buy',
                                                        mergedDf['entryRatio'] - mergedDf['stopLoss'],
                                                        np.where(
                                                                mergedDf['entrySignal'] == 'Sell',
                                                                mergedDf['entryRatio'] + mergedDf['stopLoss'],
                                                                mergedDf['stopLossRatio'].shift()

                                                        )

                        )


        # stopLossHit
        mergedDf['stopLossHit'] = None
        mask1 = pd.notna(mergedDf['entrySignal'])
        mask2 = mergedDf['signal'] == 'Buy'
        mask3 = mergedDf['ratioClose']<=mergedDf['stopLossRatio']
        mask4 = mergedDf['ratioClose'] >= mergedDf['stopLossRatio']

        for i in range(mergedDf.shape[0]):
                mergedDf['stopLossHit'] = np.where(mask1,"No", 
                                        np.where(mask2, np.where(mask3,"Yes",mergedDf['stopLossHit'].shift()), np.where(mask4,"Yes",mergedDf['stopLossHit'].shift()))

                )


        # targetRatio
        mergedDf['targetRatio'] = mergedDf['mean']



        #targetHit
        mergedDf['targetHit'] = None
        mask1 = pd.notna(mergedDf['entrySignal'])
        mask2 = mergedDf['signal'] == 'Buy'
        mask3 = mergedDf['ratioClose']>= mergedDf['targetRatio']
        mask4 = mergedDf['ratioClose'] <= mergedDf['targetRatio']

        for i in range(mergedDf.shape[0]):
                mergedDf['targetHit'] = np.where(mask1,"No", 
                                        np.where(mask2, np.where(mask3,"Yes",mergedDf['targetHit'].shift()), np.where(mask4,"Yes",mergedDf['targetHit'].shift()))

                )



        # tradeNo
        mergedDf['tradeNo'] = 0 
        for i in range(mergedDf.shape[0]):
                if pd.notna(mergedDf['entrySignal'].iloc[i]):
                        mergedDf['tradeNo'].iloc[i] = mergedDf['tradeNo'].iloc[i-1]+1
                else:
                        mergedDf['tradeNo'].iloc[i] = mergedDf['tradeNo'].iloc[i-1]
                

                mergedDf


        # exitSignal
        mergedDf['exitSignal'] = None

        mergedDf['exitSignal'] = np.where(pd.notna(mergedDf['entrySignal']), None,
                                        np.where(mergedDf['stopLossHit'] == 'Yes', 'SL_hit',
                                                np.where(mergedDf['targetHit'] == 'Yes', 'Tgt_hit', mergedDf['exitSignal'].shift())
                                                )
                                        )



        # exitPrice
        mask1 = pd.isna(mergedDf['exitSignal'].shift())
        mask2 = pd.notna(mergedDf['exitSignal'])
        mergedDf['exitPrice'] = np.where(mask1 & mask2,mergedDf[f'{company_name}Close'],np.nan )



        #PnL
        mergedDf['PnL'] = np.where(pd.notna(mergedDf['exitPrice']),
                                 np.where(mergedDf['signal'] == 'Buy', mergedDf['exitPrice'] - mergedDf['entryPrice'], mergedDf['entryPrice'] - mergedDf['exitPrice']),
                                 0
                                )


        # PnLPercent
        mergedDf['PnLPercent']= mergedDf['PnL']/mergedDf['entryPrice']*100
        filterdDf = mergedDf[(mergedDf['PnLPercent']!=0) & (pd.notna(mergedDf['PnLPercent']))]['PnLPercent']
        filterdDf =  filterdDf.reset_index().set_index('date')
        filterdDf['companyName'] = f'{company_name}'
        allStocks.append(filterdDf)

In [6]:
final_result = pd.concat(allStocks)
final_result = pd.pivot_table(final_result, values='PnLPercent', index='date', columns='companyName')
final_result

companyName,AXISBANK,HDFC,HDFCBANK,ICICIBANK,INFY,ITC,KOTAKBANK,LT,RELIANCE,TCS
date,,,,,,,,,,
2012-02-01,NaN,NaN,NaN,NaN,NaN,NaN,2.528345,NaN,NaN,NaN
2012-02-07,NaN,NaN,2.443929,NaN,NaN,NaN,-2.921394,NaN,NaN,NaN
2012-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.397709,NaN
2012-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.502898,NaN,NaN
2012-02-13,NaN,NaN,-0.859906,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.223073,NaN
2023-04-11,NaN,NaN,NaN,NaN,NaN,2.484474,NaN,NaN,NaN,NaN
2023-04-19,1.047035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
monthly_profit = final_result.resample('M').sum()
print(monthly_profit)

companyName  AXISBANK      HDFC  HDFCBANK  ICICIBANK      INFY       ITC   
date                                                                       
2012-02-29  -9.207994 -5.219645  2.180280 -11.338887  3.048564 -1.459499  \
2012-03-31   0.000000  0.000000 -2.091269   2.812354  0.000000  0.313027   
2012-04-30  -1.225343  0.823959 -1.635087   0.000000 -1.916631  0.000000   
2012-05-31   0.291643  1.380453 -2.257514  -0.023847 -1.909357  7.180848   
2012-06-30  -2.213659  1.795421  0.000000  10.438166 -0.471340  2.708810   
...               ...       ...       ...        ...       ...       ...   
2022-12-31  -1.411690 -1.349443  0.000000   2.734597 -3.138597  3.844028   
2023-01-31  -1.992032  0.691753  0.194429  -0.371216 -1.453129  1.334883   
2023-02-28   0.000000 -2.345548 -0.409546   1.151888  0.000000 -4.066246   
2023-03-31  -0.851619  1.478344  0.802768   0.000000  0.000000  0.000000   
2023-04-30  -0.011774  0.000000  0.000000   0.000000 -1.197519  2.484474   

companyName

In [8]:
# Define the start month and day of your financial year
financial_year_start_month = 4  # April
financial_year_start_day = 1    # 1st

# Define a function to determine the financial year based on the date
def get_financial_year(date):
    if date.month >= financial_year_start_month and date.day >= financial_year_start_day:
        return date.year + 1
    else:
        return date.year

# Apply the financial year function to the index to get the financial year
final_result['financial_year'] = final_result.index.map(get_financial_year)

# Calculate the annual profit by grouping by financial year and summing the profits
annual_profit = final_result.groupby('financial_year').sum()
annual_profit

companyName,AXISBANK,HDFC,HDFCBANK,ICICIBANK,INFY,ITC,KOTAKBANK,LT,RELIANCE,TCS
financial_year,,,,,,,,,,
2012,-9.207994,-5.219645,0.089011,-8.526534,3.048564,-1.146471,-3.909507,5.417615,-8.631214,-5.463215
2013,-14.345514,4.359740,-9.481372,12.089492,-11.234685,17.377688,-6.264404,-0.811512,3.401214,-2.744680
2014,17.678860,-26.348588,-1.384442,-1.399270,-2.713939,-4.995173,-15.618795,-1.936457,-15.966808,-4.670002
2015,22.659256,0.000000,7.202258,0.176257,-4.060513,-14.397644,-13.516448,8.858383,6.219642,-0.877021
2016,7.735043,6.259208,6.087224,-4.429772,-13.286980,1.502152,-3.631559,-7.076443,-1.175432,2.273141
2017,10.456301,-17.071030,-0.739580,-7.268275,-11.580290,0.046553,-22.246586,14.838750,-2.901621,-2.935649
2018,-2.553918,-2.613021,9.490195,-15.453570,-2.692164,9.788822,3.369517,-3.387171,-5.238743,-11.238330
2019,2.854727,6.638136,-7.188542,1.923433,-1.448349,11.971687,-5.077841,-10.369500,-7.586072,4.288420
2020,4.096546,-6.822410,11.894580,8.893102,18.215482,-5.314743,4.594254,-3.665142,-0.498223,7.085989
